# San Francisco Crime Classification from a top ranker

본 notebook은 Yannis Pappas 커널을 참고하여 작성했습니다. (https://www.kaggle.com/yannisp/sf-crime-analysis-prediction)

## Data Science Life Cycle
Data Science Life Cycle은 아래의 단계로 구성되어 있으며, 본 경진 대회에서도 아래의 전체 Life Cycle대로 진행할 예정입니다.
1. 데이터 품질을 향상시키기 위한 Data Wrangling
2. 탐색적 데이터 분석 (EDA)
3. 현재 Feature들을 기반으로 추가적인 Feature들을 만드는 Feature Engineering
4. (필요 시) 데이터 정규화 및 변환
5. 모델 성능 측정을 위한 훈련 데이터, 테스트 데이터 생성 및 파라미터 조정
6. 모델 선택 및 평가, 결과 예측을 위한 모델 생성

In [ ]:
import pandas as pd
from shapely.geometry import  Point
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re
import contextily as ctx
import geoplot as gplt
import lightgbm as lgb
import eli5
from eli5.sklearn import PermutationImportance
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots
import shap

In [ ]:
train = pd.read_csv('data/train.csv', parse_dates=['Dates'])
test = pd.read_csv('data/test.csv', parse_dates=['Dates'], index_col='Id')

In [ ]:
train.Dates.describe()

In [ ]:
train.shape

훈련 데이터는 2003.1.6.부터 2015.5.13.까지의 범죄를 담고 있으며, 총 9개의 features가 있습니다.

In [ ]:
train.head()

- Dates - 범죄가 일어난 일시
- Category - 범죄 유형 (이 값이 Target variable임)
- Descript - 범죄에 대한 자세한 설명
- DayOfWeek - 요일
- PdDistrict - 경찰 관할 지역 명칭
- Resolution - 범죄 해결 여부
- Address - 범죄 발생 주소
- X - 경도(Longitude)
- Y - 위도(Latitude)

In [ ]:
train.dtypes

object type, 즉 string type은 카테고리형 데이터이기 때문에 추후 인코딩이 필요합니다.

In [ ]:
train.duplicated().sum()

2323개의 중복행이 존재해 제거해줘야 합니다.

In [ ]:
def create_gdf(df):
    gdf = df.copy()
    gdf['Coordinates'] = list(zip(gdf.X, gdf.Y))
    gdf['Coordinates'] = gdf['Coordinates'].apply(Point)
    gdf = gpd.GeoDataFrame(gdf, geometry='Coordinates',
                          crs={'init': 'epsg:4326'})
    return gdf

train_gdf = create_gdf(train)

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

f, ax = plt.subplots(1, figsize=(9,9))
ax = world.plot(color='white', edgecolor='black', axes=ax)
train_gdf.plot(ax=ax, color='red');

캘리포니아 지역의 범죄에 대한 데이터인데 엉뚱한 곳에 찍혀 있는 데이터가 있습니다. 어떤 데이터인지 확인해보겠습니다.

In [ ]:
train_gdf[train_gdf.Y > 70]

In [ ]:
train_gdf[train_gdf.Y > 70].count()[0]

총 67개의 데이터의 좌표가 잘못되어 있습니다.
우선, 중복행은 제거를 시켜줍니다. 그리고 67개의 outlier는 평균값으로 대체합니다.

In [ ]:
train.drop_duplicates(inplace=True)

In [ ]:
train.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
test.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

In [ ]:
imputer = SimpleImputer(strategy='mean')

In [ ]:
for district in train['PdDistrict'].unique():
    train.loc[train['PdDistrict']==district, ['X', 'Y']] = imputer.fit_transform(
        train.loc[train['PdDistrict']==district, ['X', 'Y']])
    # fit은 train 데이터로 해주었기때문에 transform만 적용
    test.loc[test['PdDistrict']==district, ['X', 'Y']] = imputer.transform(
        test.loc[test['PdDistrict']==district, ['X', 'Y']])    
    
train_gdf = create_gdf(train)

### 날짜와 요일

In [ ]:
train['Date'] = train['Dates'].dt.date
train['Year'] = train['Dates'].dt.year
train['Month'] = train['Dates'].dt.month
train['Day'] = train['Dates'].dt.day
train['Hour'] = train['Dates'].dt.hour

In [ ]:
year_series = train.groupby('Year').count().iloc[:, 0]
g = sns.barplot(x=year_series.index, y=year_series)
g.set_xticklabels(g.get_xticklabels(), rotation=45);

연도에 따른 범죄수입니다. 2003년부터 2014년까지는 범죄수가 거의 유사했지만, 2015년에 급감한 수치를 보입니다.

In [ ]:
month_series = train.groupby('Month').count().iloc[:, 0]
sns.barplot(x=month_series.index, y=month_series);

월에 따른 범죄수입니다. 8월, 12월에 범죄수가 가장 적었고, 5월, 10월에 가장 많았습니다. 날씨가 안 좋을 때 (더울 때 혹은 추울 때)는 범죄도 적고, 날씨가 좋을 때는 범죄도 많다는 것을 알 수 있습니다.

In [ ]:
hour_series = train.groupby('Hour').count().iloc[:, 0]
sns.barplot(x=hour_series.index, y=hour_series);

시간에 따른 범죄수입니다. 예상대로 모두가 잠든 새벽 시간에 범죄가 가장 적고, 18시가 가장 많습니다. 아침부터 18시까지 점차 증가하는 추세를 보이는데 12시에 유독 많은 것을 볼 수 있습니다. 점심 시간, 저녁 시간 등과 관련이 있지 않나 추측해봅니다.

In [ ]:
palette = sns.color_palette()

plt.figure(figsize=(10, 6))
date_count = train.groupby('Date').count().iloc[:, 0]
sns.kdeplot(data=date_count, shade=True)
plt.axvline(x=date_count.median(), ymax=0.95, linestyle='--', color=palette[1])
plt.annotate('Median ' + str(date_count.median()),
             xy=(date_count.median(), 0.004),
             xytext=(200, 0.005),
             arrowprops=dict(arrowstyle='->', color=palette[1], shrinkB=10))
plt.title('Distribution of number of incidents per day',fontdict={'fontsize':16})
plt.xlabel('Crime Incidents')
plt.ylabel('Density')
plt.legend().remove()
plt.show()

하루동안 발생하는 범죄 건수는 정규 분포를 그리고 있고, 그 중앙값은 389회입니다. 

In [ ]:
weekday_series = train.groupby('DayOfWeek').count().iloc[:,0]
weekday_series = weekday_series.reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
    'Sunday'])

with sns.axes_style("whitegrid"):
    f, ax = plt.subplots(1, figsize=(10, 6))
    sns.barplot(
        weekday_series.index, (weekday_series.values / weekday_series.values.sum()) * 100,
        palette=cm.ScalarMappable(cmap='Blues').to_rgba(weekday_series.values))

plt.title('Incidents per Weekday', fontdict={'fontsize':16})
plt.xlabel('Weekday')
plt.ylabel('Percent of Incients (%)');

금요일에 범죄 건수가 가장 많고, 수요일, 토요일, 목요일 등이 그 뒤를 이었습니다

### Category

In [ ]:
category_counts = train.groupby('Category').count().iloc[:, 0].sort_values(ascending=False)

In [ ]:
category_counts

In [ ]:
# OTHER OFFENSES를 제일 아래 두기 위해
category_counts = category_counts.reindex(
    np.append(np.delete(category_counts.index, 1), 'OTHER OFFENSES'))

In [ ]:
with sns.axes_style("whitegrid"):
    f, ax = plt.subplots(1, figsize=(10, 10))
    sns.barplot(
        category_counts.values / category_counts.values.sum() * 100,
        category_counts.index,
        orient='h',
        palette='Blues_d')
plt.title('Incidents per Crime Category', fontdict={'fontsize': 16})
plt.xlabel('Incidents (%)');

절도의 비율이 가장 큽니다.

### Police District

샌프란시스코의 Police Distirct를 불러와 train 데이터와 merging시킵니다.

In [ ]:
# Downloading the shapefile of the area 
url = 'https://data.sfgov.org/api/geospatial/wkhw-cjsf?method=export&format=Shapefile'
with urllib.request.urlopen(url) as response, open('pd_data.zip', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
    
# Unzipping it
with zipfile.ZipFile('pd_data.zip', 'r') as zip_ref:
    zip_ref.extractall('pd_data')
    
# Loading to a geopandas dataframe
for filename in os.listdir('./pd_data/'):
    if re.match(".+\.shp", filename):
        pd_districts = gpd.read_file('./pd_data/'+filename)
        break
        
# Merging our train dataset with the geo-dataframe
pd_districts = pd_districts.merge(
    train.groupby('PdDistrict').count().iloc[:, [0]].rename(
        columns={'Dates': 'Incidents'}),
    left_on='district',
    right_index=True)

# Transforming the coordinate system to Spherical Mercator for
# compatibility with the tiling background
pd_districts = pd_districts.to_crs({'init': 'epsg:3857'})

# Calculating the incidents per day for every district
train_days = train.groupby('Date').count().shape[0]
pd_districts['inc_per_day'] = pd_districts.Incidents/train_days

각 district별 하루 평균 범죄 발생 건수를 시각화합니다.

In [ ]:
# Ploting the data
fig, ax = plt.subplots(figsize=(10, 10))
pd_districts.plot(
    column='inc_per_day',
    cmap='Reds',
    alpha=0.6,
    edgecolor='r',
    linestyle='-',
    linewidth=1,
    legend=True,
    ax=ax);

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
pd_districts.plot(
    column='inc_per_day',
    cmap='Reds',
    alpha=0.6,
    edgecolor='r',
    linestyle='-',
    linewidth=1,
    legend=True,
    ax=ax);

def add_basemap(ax, zoom, url='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'):
    """Function that add the tile background to the map"""
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, url=url)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    # restore original x/y limits
    ax.axis((xmin, xmax, ymin, ymax))

# Adding the background
add_basemap(ax, zoom=11, url=ctx.sources.ST_TONER_LITE)

# Adding the name of the districts
for index in pd_districts.index:
    plt.annotate(
        pd_districts.loc[index].district,
        (pd_districts.loc[index].geometry.centroid.x,
         pd_districts.loc[index].geometry.centroid.y),
        color='#353535',
        fontsize='large',
        fontweight='heavy',
        horizontalalignment='center'
    )

ax.set_axis_off()
plt.show()

### Address

범죄별로 발생 지역을 시각화해줍니다.

In [ ]:
crimes = train['Category'].unique().tolist()
crimes.remove('TREA')

pd_districts = pd_districts.to_crs({'init': 'epsg:4326'})

# geometry containing the union of all geometries 
sf_land = pd_districts.unary_union

In [ ]:
sf_land

In [ ]:
sf_land = gpd.GeoDataFrame(gpd.GeoSeries(sf_land), crs={'init':'epsg:4326'})
sf_land = sf_land.rename(columns={0:'geometry'}).set_geometry('geometry')

In [ ]:
fig, ax = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(12,12))
for i, crime in enumerate(np.random.choice(crimes, size=9, replace=False)):
    data = train_gdf.loc[train_gdf['Category'] == crime]
    ax = fig.add_subplot(3, 3, i+1)
    gplt.kdeplot(data,
                shade=True,
                shade_lowest=False, # False일 때, 0에 가까운 빈도를 가진 구역은 진하기를 표현하지 않음
                clip=sf_land.geometry,# 주어진 구역만 시각화
                cmap='Reds',
                ax=ax)
    gplt.polyplot(sf_land, ax=ax)
    ax.set_title(crime)
plt.suptitle('Geographic Density of Diffenrent Crimes')
fig.tight_layout(rect=[0, 0.03, 1, 0.95]);

시간대별 주요 범죄 발생 건수를 시각화해봅니다.

In [ ]:
# as_index=False: Hour, Date, Category를 index로 지정하지 않음
data = train.groupby(['Hour', 'Date', 'Category'],
                    as_index=False).count().iloc[:, :4]
data

In [ ]:
data.rename(columns={'Dates': 'Incidents'}, inplace=True)
data = data.groupby(['Hour', 'Category'], as_index=False).mean()
data = data.loc[data['Category'].isin(
    ['ROBBERY', 'GAMBLING', 'BUGLARY', 'ARSON', 'PROSTITUTION'])]

data

In [ ]:
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(14,4))
ax = sns.lineplot(data=data, x='Hour', y='Incidents', hue='Category')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=6)
plt.suptitle('Average number of incidents per hour')
fig.tight_layout(rect=[0, 0, 1, 0.95])

도박은 새벽부터 다음날 아침까지 많이 발생합니다. 매춘은 저녁시간부터 밤새 많이 발생하며, 이른 아침부터 저녁까지 꾸준히 증가하는 것을 볼 수 있습니다.

### Naive Prediction

In [ ]:
naive_vals = train.groupby('Category').count().iloc[:, 0] / train.shape[0]
n_rows = test.shape[0]

naive_vals

In [ ]:
submission = pd.DataFrame(
    np.repeat(np.array(naive_vals), n_rows).reshape(39, n_rows).transpose(),
    columns=naive_vals.index)

In [ ]:
submission

train 데이터에서 각 Category의 비율을 test 데이터에 그대로 넣은 것입니다. 모든 row의 값은 동일합니다. submission했을 때 score는 2.68015입니다.

### Methodology

#### Data Wranling

2323개의 중복행과 67개의 잘못된 위도, 경도 값이 있었습니다. 이는 이미 위에서 처리했습니다.

#### Feature Engineering

In [ ]:
def feature_engineering(data):
    # object type -> datetime type으로
    data['Date'] = pd.to_datetime(data['Dates'].dt.date) 
    # timedelta type -> int type으로
    data['n_days'] = (
        data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    data['Day'] = data['Dates'].dt.day
    data['DayOfWeek'] = data['Dates'].dt.weekday
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['Block'] = data['Address'].str.contains('block', case=False)
    
    data.drop(columns=['Dates', 'Date', 'Address'], inplace=True)
    
    return data

In [ ]:
train = feature_engineering(train)
train.drop(columns=['Descript', 'Resolution'], inplace=True)
test = feature_engineering(test)
train.head()

Descript와 Resolution은 train 데이터에만 있으므로 예측하는데 필요없는 feature입니다. 분석을 위한 feature가 되기 위해서는 train 데이터에도 test 데이터에도 모두 존재해야 합니다.

#### Feature Scaling

Tree-based model에서는 feature scaling이 따로 필요없습니다.

#### Feature Selection

Feature engineering 후 총 11개의 feature가 남았습니다. 

In [ ]:
PdDistrict_le = LabelEncoder()
train['PdDistrict'] = PdDistrict_le.fit_transform(train['PdDistrict'])
test['PdDistrict'] = PdDistrict_le.transform(test['PdDistrict'])

Category_le = LabelEncoder()
y = Category_le.fit_transform(train.pop('Category'))

train_X, val_X, train_y, val_y = train_test_split(train, y)
model = LGBMClassifier(objective='multiclass', num_class=39).fit(train_X, train_y)

# 하나의 column을 섞어 성능을 구했을 때 성능 감소량이 그 feature의 중요도임
perm = PermutationImportance(model).fit(val_X, val_y)
eli5.show_weights(perm, feature_names=val_X.columns.tolist())

Permutation Importance: 이미 훈련된 모델에서 어떤 feature가 중요한지 판단하는 지표입니다. 하나의 feature를 섞고, 나머지 feature는 그대로 둔 채 성능을 평가합니다. 성능의 감소치만큼 해당 feature가 중요하다는 것을 의미합니다. 같은 방식으로 모든 feature를 섞어가며 해당 feature의 중요도를 측정합니다. 위 도표는 feature의 중요도를 순서대로 나타낸 표입니다.
결론적으로 Permutation Importance는 특정 feature가 예측 정확도에 얼마나 많은 영향을 미치는가 판단할 수 있는 지표입니다. 하지만 예측을 더 정확하게 하는 방향으로 영향을 미치는지 더 부정확하게 하는 방향으로 영향을 미치는지 (즉, direction)은 알 수가 없습니다.

## Building Model

In [ ]:
# Loading the data
train = pd.read_csv('./data/train.csv', parse_dates=['Dates'])
test = pd.read_csv('./data/test.csv', parse_dates=['Dates'], index_col='Id')

# Data cleaning
train.drop_duplicates(inplace=True)
train.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
test.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

imp = SimpleImputer(strategy='mean')

for district in train['PdDistrict'].unique():
    train.loc[train['PdDistrict'] == district, ['X', 'Y']] = imp.fit_transform(
        train.loc[train['PdDistrict'] == district, ['X', 'Y']])
    test.loc[test['PdDistrict'] == district, ['X', 'Y']] = imp.transform(
        test.loc[test['PdDistrict'] == district, ['X', 'Y']])
train_data = lgb.Dataset(
    train, label=y, categorical_feature=['PdDistrict'], free_raw_data=False)

# Feature Engineering
train = feature_engineering(train)
train.drop(columns=['Descript','Resolution'], inplace=True)
test = feature_engineering(test)

# Encoding the Categorical Variables
le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])

le2 = LabelEncoder()
X = train.drop(columns=['Category'])
y= le2.fit_transform(train['Category'])

# Creating the model
train_data = lgb.Dataset(
    X, label=y, categorical_feature=['PdDistrict'])

params = {'boosting':'gbdt',
          'objective':'multiclass',
          'num_class':39,
          'max_delta_step':0.9,
          'min_data_in_leaf': 21,
          'learning_rate': 0.4,
          'max_bin': 465,
          'num_leaves': 41
         }

bst = lgb.train(params, train_data, 100)

predictions = bst.predict(test)

# Submitting the results
submission = pd.DataFrame(
    predictions,
    columns=le2.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),
    index=test.index)
submission.to_csv(
    'LGBM_final.csv', index_label='Id')